<a href="https://colab.research.google.com/github/ShlokM08/CSE508_Winter2024_A2_2021421./blob/main/IRASSIGNMENT2f.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#IMPORT DRIVE
from google.colab import drive
drive.mount('/content/drive')
!pip install python-docx nltk


Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 4.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from torchvision import models, transforms
from PIL import Image
import requests
from io import BytesIO
import pickle
import numpy as np
import os

class ImageProcessor:
    def __init__(self):
        self.transform_pipeline = transforms.Compose([
            transforms.Resize(299),
            transforms.CenterCrop(299),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        self.model = models.inception_v3(pretrained=True)
        self.model.fc = torch.nn.Identity()
        self.model.eval()

    def fetch_and_process(self, url):
        try:
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                with Image.open(BytesIO(response.content)).convert("RGB") as img:
                    img_tensor = self.transform_pipeline(img).unsqueeze(0)
                    with torch.no_grad():
                        features = self.model(img_tensor)
                return features.squeeze().numpy()
            print(f"Failed to download image from {url}")
        except Exception as e:
            print(f"Error processing image from {url}: {e}")
        return None

class DatasetManager:
    def __init__(self, dataset_path, feature_path):
        self.dataset_path = dataset_path
        self.feature_path = feature_path
        self.data = pd.read_csv(dataset_path)
        self.image_processor = ImageProcessor()
        self.data.fillna('', inplace=True)

    def process_images(self):
        if os.path.exists(self.feature_path):
            print("Features already exist. Loading...")
            return

        feature_dict = {}
        for index, row in self.data.iterrows():
            images = eval(row['Image'])
            features = [self.image_processor.fetch_and_process(url) for url in images if url]
            feature_dict[index] = [feature for feature in features if feature is not None]

        with open(self.feature_path, 'wb') as f:
            pickle.dump(feature_dict, f)
        print("Image processing complete.")

    def load_features(self):
        with open(self.feature_path, 'rb') as f:
            return pickle.load(f)

if __name__ == "__main__":
    dataset_path = '/content/drive/MyDrive/A2_Data.csv'
    feature_path = '/content/drive/MyDrive/image_features.pkl'

    manager = DatasetManager(dataset_path, feature_path)
    manager.process_images()
    features = manager.load_features()
    print("Feature extraction is done.")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Features already exist. Loading...
Feature extraction is done.


In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
from collections import Counter
import math
import pickle


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Simplified text preprocessing function
def preprocess_text(text):
    lower_text = text.lower()
    cleaned_text = ''.join(char for char in lower_text if char not in string.punctuation)
    tokens = word_tokenize(cleaned_text)
    filtered_tokens = [word for word in tokens if word not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    return lemmatized_tokens

# Compute Term Frequency
def compute_tf(document):
    counts = Counter(document)
    return {word: count / len(document) for word, count in counts.items()}

# Compute Inverse Document Frequency
def compute_idf(word, corpus):
    return math.log10(len(corpus) / sum(1 for doc in corpus if word in doc))

# Compute TF-IDF for the entire corpus
def compute_tfidf(corpus):
    documents_tf = [compute_tf(doc) for doc in corpus]
    idf_values = {word: compute_idf(word, corpus) for doc in corpus for word in doc}
    tfidf_scores = [{word: tf * idf_values[word] for word, tf in doc_tf.items()} for doc_tf in documents_tf]
    return tfidf_scores

if __name__ == "__main__":
    # Specify file locations
    dataset_path = '/content/drive/MyDrive/A2_Data.csv'
    tfidf_path = '/content/drive/MyDrive/IR_ASSIGNMENT_2/tfidf.pkl'

    # Load dataset
    df = pd.read_csv(dataset_path)
    df['Review Text'].fillna('', inplace=True)

    # Preprocess reviews
    df['Processed Reviews'] = df['Review Text'].apply(preprocess_text)
    corpus = df['Processed Reviews'].tolist()

    # Compute and save TF-IDF scores
    tfidf_scores = compute_tfidf(corpus)
    with open(tfidf_path, 'wb') as file:
        pickle.dump(tfidf_scores, file)

    print("TF-IDF scores have been computed and saved.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


TF-IDF scores have been computed and saved.


In [5]:
import pickle
import numpy as np
from scipy.spatial.distance import cosine
import pandas as pd
import nltk
import string
from collections import Counter
import math
import os
import requests
from PIL import Image
from torchvision import models, transforms
from io import BytesIO
import torch
from requests.exceptions import ConnectionError, HTTPError, Timeout

# Preprocessing dependencies
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Globals
stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = nltk.stem.WordNetLemmatizer()
transform_pipeline = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
model = models.inception_v3(weights=models.Inception_V3_Weights.IMAGENET1K_V1)
model.fc = torch.nn.Identity()
model.eval()

# Preprocessing Functions
def preprocess_text(text):
    text = text.lower().translate(str.maketrans('', '', string.punctuation))
    words = nltk.word_tokenize(text)
    words = [word for word in words if word not in stopwords]
    return [lemmatizer.lemmatize(word) for word in words]

def preprocess_image(image_url, max_tries=3):
    for attempt in range(max_tries):
        try:
            response = requests.get(image_url, timeout=10)
            if response.status_code == 200:
                img = Image.open(BytesIO(response.content)).convert('RGB')
                img_transformed = transform_pipeline(img).unsqueeze(0)
                with torch.no_grad():
                    features = model(img_transformed)
                return torch.nn.functional.normalize(features, p=2, dim=1).cpu().numpy()
        except (ConnectionError, HTTPError, Timeout) as e:
            print(f"Attempt {attempt + 1} failed for {image_url}: {e}")
    print(f"Failed to process image {image_url} after {max_tries} attempts.")
    return None

# TF and Cosine Similarity Functions
def calculate_tf(document):
    tf = Counter(document)
    return {word: count / float(len(document)) for word, count in tf.items()}

def cosine_similarity(v1, v2):
    return 1 - cosine(v1, v2)

def find_similar_images(input_features, image_feature_dict, top_n=3):
    similarities = [(index, cosine_similarity(np.squeeze(input_features), np.squeeze(features)))
                    for index, feature_list in image_feature_dict.items()
                    for features in feature_list]
    return sorted(similarities, key=lambda x: x[1], reverse=True)[:top_n]

def find_similar_reviews(input_tf, tfidf_list, vocab, top_n=3):
    input_vector = np.array([input_tf.get(term, 0) for term in vocab])
    similarities = [(index, cosine_similarity(input_vector, np.array([tfidf_dict.get(term, 0) for term in vocab])))
                    for index, tfidf_dict in enumerate(tfidf_list)]
    return sorted([sim for sim in similarities if sim[1] >= 0], key=lambda x: x[1], reverse=True)[:top_n]

def load_data(image_features_path, tfidf_path):
    with open(image_features_path, 'rb') as f:
        img_features = pickle.load(f)
    with open(tfidf_path, 'rb') as f:
        tf_scores = pickle.load(f)
    return img_features, tf_scores

# Main
def main(image_url, review_text, img_features_path, tfidf_path, results_path):
    img_features, tf_scores = load_data(img_features_path, tfidf_path)
    vocab = sorted(set(term for tfidf_dict in tf_scores for term in tfidf_dict.keys()))

    processed_img = preprocess_image(image_url)
    processed_review = preprocess_text(review_text)
    review_tf = calculate_tf(processed_review)

    similar_images = find_similar_images(processed_img, img_features)
    similar_reviews = find_similar_reviews(review_tf, tf_scores, vocab)

    print("Top 3 similar images:", similar_images)
    print("Top 3 similar reviews:", similar_reviews)

    # Save results
    with open(results_path, 'wb') as f:
        pickle.dump({'imgsSim': similar_images, 'revsSim': similar_reviews}, f)
    print("Similar Results have been saved.")

if __name__ == "__main__":
    img_url_input = input("Enter the image URL: ")
    review_text_input = input("Enter the review text: ")
    image_features_path = '/content/drive/MyDrive/IR_ASSIGNMENT_2/image_features.pkl'
    tfidf_path = '/content/drive/MyDrive/IR_ASSIGNMENT_2/tfidf.pkl'
    results_path = '/content/drive/MyDrive/IR_ASSIGNMENT_2/sim_results.pkl'

    main(img_url_input, review_text_input, image_features_path, tfidf_path, results_path)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 111MB/s] 


Enter the image URL: https://images-na.ssl-images-amazon.com/images/I/71bztfqdg+L._SY88.jpg
Enter the review text: I have been using Fender locking tuners for about five years on various strats and teles. Definitely helps with tuning stability and way faster to restring if there is a break.


/usr/local/lib/python3.10/dist-packages/scipy/spatial/distance.py:636: RuntimeWarning: invalid value encountered in scalar divide
  dist = 1.0 - uv / np.sqrt(uu * vv)


Top 3 similar images: [(655, 0.7960432767868042), (237, 0.7892007827758789), (939, 0.777145266532898)]
Top 3 similar reviews: [(828, 1), (758, 0.9660084816861149), (622, 0.30610582916494966)]
Similar Results have been saved.


In [ ]:
#q4
import pandas as pd
import numpy as np
import pickle

# Specify the path to the stored similarity results
similarity_results_path = '/content/drive/MyDrive/IR_ASSIGNMENT_2/similarity_outcomes.pkl'

# Utility to fetch results from the designated pickle file
def retrieve_results(file_path):
    with open(simResultPath, 'rb') as data_file:
        results = pickle.load(data_file)
    return results['imgsSim'], results['revsSim']

# Generate composite scores from similarities and sort them
def compile_rankings(image_matches, text_matches):
    composite_list = []

    for img_match, txt_match in zip(image_matches, text_matches):
        image_idx, img_similarity = img_match
        text_idx, txt_similarity = txt_match

        # Average of similarities as composite score
        average_score = (img_similarity + txt_similarity) / 2
        composite_list.append((image_idx, text_idx, average_score))

    # Sorting the composite scores in descending order
    return sorted(composite_list, key=lambda item: item[2], reverse=True)[:3]

# Presentation of the ranked results
def present_ranked_outcomes(sorted_results):
    print("Summary of Composite Rankings:")
    for ranking, (img_id, rev_id, comp_score) in enumerate(sorted_results, start=1):
        print(f"Position: {ranking}, Image Ref: {img_id}, Review Ref: {rev_id}, Combined Score: {comp_score:.4f}")

# Main execution flow
if __name__ == "__main__":
    image_sims, review_sims = retrieve_results(similarity_results_path)
    sorted_composites = compile_rankings(image_sims, review_sims)
    present_ranked_outcomes(sorted_composites)

    # Storing the compiled and ranked results for future reference
    ranking_storage_path = '/content/drive/MyDrive/IR_ASSIGNMENT_2/compiled_rankings.pkl'
    with open(ranking_storage_path, 'wb') as storage:
        pickle.dump(sorted_composites, storage)
    print("The compiled rankings are now stored.")



Summary of Composite Rankings:
Position: 1, Image Ref: 655, Review Ref: 758, Combined Score: 0.8810
Position: 2, Image Ref: 237, Review Ref: 622, Combined Score: 0.5477
Position: 3, Image Ref: 939, Review Ref: 439, Combined Score: 0.4756
The compiled rankings are now stored.


In [2]:
import pandas as pd
import numpy as np
import pickle

# Load the necessary data
data_location = '/content/drive/MyDrive/A2_Data.csv'
similarity_results_location = '/content/drive/MyDrive/IR_ASSIGNMENT_2/sim_results.pkl'
ranking_results_location = '/content/drive/MyDrive/IR_ASSIGNMENT_2/rank_results.pkl'

df = pd.read_csv(data_location)
with open(similarity_results_location, 'rb') as file:
    similarity_results = pickle.load(file)
with open(ranking_results_location, 'rb') as file:
    ranking_results = pickle.load(file)

# Improved function for composite result calculation and display
def calculate_and_display_composite_results(data_frame, similarity_results, ranking_results):
    print("\n" + "-" * 100)
    sections = ["IMAGE RETRIEVAL", "TEXT RETRIEVAL"]
    for section in sections:
        print(f"\nUSING {section}\n" + "-" * 100)

        for i, (img_index, review_index, comp_score) in enumerate(ranking_results, start=1):
            # Adjusting retrieval based on section
            if section == "IMAGE RETRIEVAL":
                url_img = data_frame.iloc[img_index]['Image']
                text_review = data_frame.iloc[review_index]['Review Text']
            else:  # Correcting the mistake for text retrieval results
                url_img = data_frame.iloc[img_index]['Image']  # Use img_index for consistency in example
                text_review = data_frame.iloc[img_index]['Review Text']  # Adjusted to use img_index

            img_sim = similarity_results['imgsSim'][i-1][1] if i-1 < len(similarity_results['imgsSim']) else 0
            rev_sim = similarity_results['revsSim'][i-1][1] if i-1 < len(similarity_results['revsSim']) else 0

            # Displaying results with a visually different structure
            print(f"Result #{i}:")
            print(f"Image URL: {url_img}\nReview: {text_review}\nImage Similarity: {img_sim:.4f}, Text Similarity: {rev_sim:.4f}\nComposite Score: {comp_score:.4f}\n")

    # Composite similarity score calculations
    img_similarities = [sim[1] for sim in similarity_results['imgsSim']]
    text_similarities = [sim[1] for sim in similarity_results['revsSim']]
    img_composite_score = np.mean(img_similarities)
    text_composite_score = np.mean(text_similarities)
    final_composite_score = (img_composite_score + text_composite_score) / 2

    # Displaying final scores with a visually different output
    print("-" * 100)
    print(f"Composite Scores:\nImages: {img_composite_score:.4f}, Text: {text_composite_score:.4f}\nFinal Composite: {final_composite_score:.4f}\n" + "-" * 100)

# Execute the updated function
calculate_and_display_composite_results(df, similarity_results, ranking_results)



----------------------------------------------------------------------------------------------------

USING IMAGE RETRIEVAL
----------------------------------------------------------------------------------------------------
Result #1:
Image URL: ['https://images-na.ssl-images-amazon.com/images/I/719-SDMiOoL._SY88.jpg']
Review: I have been using Fender locking tuners for about five years on various strats and teles. Definitely helps with tuning stability and way faster to restring if there is a break.
Image Similarity: 0.7960, Text Similarity: 0.9660
Composite Score: 0.8810

Result #2:
Image URL: ['https://images-na.ssl-images-amazon.com/images/I/71vT2-nW7-L._SY88.jpg']
Review: I went from fender chrome non-locking to fender gold locking. It made my guitar look beautiful and play beautiful. I think locking tuners are the way to go. If you are new to locking tuners look on YouTube for instructions.
Image Similarity: 0.7892, Text Similarity: 0.3061
Composite Score: 0.5477

Result #3:
Im